In [1]:
import asyncio
import importlib
import random
from functools import partial

from tqdm.auto import tqdm

from cuery import asy


In [2]:
# Deterministic retry test: make each task fail FAIL_FIRST times then succeed
attempts: dict[int, int] = {}
FAIL_FIRST = 2  # number of initial failures we force per task


async def sometimes_fails(x: int) -> int:
    attempt = attempts.get(x, 0)
    attempts[x] = attempt + 1
    # small sleep to exercise timeout/semaphore logic without slowing too much
    if attempt < FAIL_FIRST:
        raise ValueError(f"Injected failure attempt {attempt + 1} for task {x}")
    await asyncio.sleep(1 + random.random() * 1.1)
    return x * 2


async def progress_callback(status: dict):
    print("Progress update:", status["n"], "/", status["total"])


pbar = tqdm(total=10)
params = [{"x": i} for i in [3, 5, 2, 8, 1, 4, 6, 7, 9, 10]]


policies = {
    "timeout": 1.9,
    "n_concurrent": 3,
    "retries": 3,
    "wait_max": 10,
    "fallback": -1,
    "pbar": pbar,
    "progress_callback": progress_callback,
    "min_iters": 2,
}

coros = asy.all_with_policies(sometimes_fails, kwds=params, policies=policies)

results = await asyncio.gather(*coros)
print("Results:", results)
print("Attempts per task:", attempts)

expected_attempts = FAIL_FIRST + 1
all_ok = all(a == expected_attempts for a in attempts.values())
print("All tasks retried expected times:", all_ok)

assert all_ok, (
    f"Unexpected attempt distribution: {attempts} (expected each == {expected_attempts})"
)
results

  0%|          | 0/10 [00:00<?, ?it/s]

2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 3
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 5
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 2
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 8
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 1
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 4
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 6
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 7
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 9
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 1 failed: Injected failure attempt 1 for task 10
2025-09-17 17:19:54 [cuery|INFO] Retry attempt 2 

Progress update: 2 / 10


2025-09-17 17:19:57 [cuery|INFO] Retry attempt 3 failed: Awaitable timed out!
2025-09-17 17:19:57 [cuery|WARNING] Coroutine fallback: RetryError[<Future at 0x11628e710 state=finished raised TimeoutError>]


Progress update: 4 / 10
Progress update: 6 / 10


2025-09-17 17:19:59 [cuery|INFO] Retry attempt 3 failed: Awaitable timed out!
2025-09-17 17:19:59 [cuery|WARNING] Coroutine fallback: RetryError[<Future at 0x1162d9710 state=finished raised TimeoutError>]


Progress update: 8 / 10


2025-09-17 17:20:00 [cuery|INFO] Retry attempt 3 failed: Awaitable timed out!
2025-09-17 17:20:00 [cuery|WARNING] Coroutine fallback: RetryError[<Future at 0x1162da950 state=finished raised TimeoutError>]


Progress update: 10 / 10
Results: [6, -1, 4, 16, -1, 8, -1, 14, 18, -1]
Attempts per task: {3: 3, 5: 3, 2: 3, 8: 3, 1: 3, 4: 3, 6: 3, 7: 3, 9: 3, 10: 3}
All tasks retried expected times: True


[6, -1, 4, 16, -1, 8, -1, 14, 18, -1]